In [ ]:
# import libraries
import numpy as np
import pandas as pd

In [ ]:
# Load the credits and movies datasets
credits=pd.read_csv("./data/tmdb_5000_credits.csv")
movies=pd.read_csv("./data/tmdb_5000_movies.csv")

In [ ]:
movies.head()

In [ ]:
movies.dtypes

In [ ]:
credits.head(1)

In [ ]:
credits['crew']

In [ ]:
credits.dtypes

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
movies.head()

In [ ]:
#genres
#id
#title
#overview(most import according to content)
#cast
#crew
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.isnull().sum()

In [ ]:
#as overview is most important column and if we have missing values, either we have to manually enter values or drop them.so i am dropping this

movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies['genres']

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast # used to convert string in to list

def convert(obj):
    L =[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres']=movies['genres'].apply(convert) #convert function is applied to each row of genres column


In [ ]:
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
movies.iloc[0].cast

In [ ]:
# function to extract the first three actors from the 'cast' column
def actor(obj):
    L =[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast']=movies['cast'].apply(actor)

In [ ]:
movies.head()

In [ ]:
movies.iloc[0].crew

In [ ]:
# function to fetch the director from the 'crew' column
def fetch_director(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job'] =='Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies['crew'].isnull().sum()

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
# tokenize the 'overview' column because we have to convert the string to list
movies['overview']=movies['overview'].apply(lambda x:x.split()) # string to list

In [ ]:
movies.head()

In [ ]:
# remove spaces from the genre, cast, and crew lists
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


In [ ]:
movies.head()

In [ ]:
new_df=movies[['movie_id','title','tags']]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))  # use " ".join to convert list to string with spaces

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
# a function to apply stemming to text, why? because we have to convert the words to their root form
def stem(text):
    y=[]
    
    for i in text.split(): # so we will stem every word in the list
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
# apply stemming to the 'tags' column
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
# convert text to lowercase in the 'tags' column
new_df['tags']=new_df['tags'].apply(lambda x :x.lower())

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'][1]

In [ ]:
# Text Vectorization

# so now we have to convert tags in to vector and then all movie will become a vector.so total 45000 vectors will be there.
# so to recommend , we have to select the closest vectors
# I am using bag of words for now
# remove stopwords

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors.shape

In [ ]:
vectors[0] # so many zero because we have 5000 words and our tag has not that much words

In [ ]:
cv.get_feature_names_out()# now we have problem with same words like accept ,accepted,acceptence,accepts .we need to apply stemming.

In [ ]:
# what stemming will do that ['loved','loving','love']=> will get output as ['love','love','love']
# after that we have to find cosine distance

In [ ]:
# calculate cosine similarity between vectors
from sklearn.metrics.pairwise import cosine_similarity # it is between 0 and 1

In [ ]:
vectors #every row of the array has 5000 columns

In [ ]:
# calculate cosine similarity matrix
similarity=cosine_similarity(vectors)

In [ ]:
similarity.shape 

In [ ]:
similarity

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key = lambda x: x[1])[1:6]

In [ ]:
index = new_df[new_df['title'] == 'Avatar'].index[0]

distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])

In [ ]:
distances

In [ ]:
# a function to recommend similar movies
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:# i is the tuple and 
        print(new_df.iloc[i[0]].title)# i[0] is the index of every tuple

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Batman Begins')

In [ ]:
# Save the new dataset and similarity matrix to pickle files
import pickle

pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))